In [ ]:
import cv2
import mediapipe as mp
import time
import random

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils
handLmsStyle = mpDraw.DrawingSpec(color=(0, 0, 255), thickness=3)
handConStyle = mpDraw.DrawingSpec(color=(0, 255, 0), thickness=5)
pTime = 0
cTime = 0

# Set up game variables
disc_radius = 30
disc_x = random.randint(disc_radius, 640 - disc_radius)
disc_y = 480 - disc_radius
score = 0

while True:
    ret, img = cap.read()
    if ret:
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(imgRGB)

        imgHeight = img.shape[0]
        imgWidth = img.shape[1]

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:
                for i, lm in enumerate(handLms.landmark):
                    xPos = int(lm.x * imgWidth)
                    yPos = int(lm.y * imgHeight)

                    # Check if hand is below the disc
                    if abs(xPos - disc_x) < disc_radius and yPos > disc_y - disc_radius:
                        # Update disc position
                        disc_x = random.randint(disc_radius, 640 - disc_radius)
                        disc_y = 480 - disc_radius
                        score += 1

        # Draw the disc
        cv2.circle(img, (disc_x, disc_y), disc_radius, (255, 0, 0), cv2.FILLED)

        # Display game info
        cv2.putText(img, f"Score: {score}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, f"FPS : {int(fps)}", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

        cv2.imshow('Disc Game', img)

    if cv2.waitKey(1) == ord('q'):
        break
